In [16]:
import asyncio
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta, date
from zoneinfo import ZoneInfo
from alpaca.data.live.stock import StockDataStream
from alpaca.data.requests import StockBarsRequest, StockQuotesRequest
from alpaca.data.enums import DataFeed
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.stream import TradingStream
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, StopOrderRequest, LimitOrderRequest, StopLimitOrderRequest, TrailingStopOrderRequest, GetCalendarRequest
from alpaca.trading.enums import OrderSide, OrderStatus, TimeInForce, OrderType
from alpaca.trading.models import Order, TradeUpdate
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.enums import Adjustment
from types import SimpleNamespace
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict, Set
from collections import defaultdict

from alpaca.data.models import Bar

from IPython.display import clear_output


# Importing the modules themselves for reloading
import TradePosition
import TouchArea
import LiveTrader

# Reloading the modules to apply any changes
import importlib
importlib.reload(TradePosition)
importlib.reload(TouchArea)
importlib.reload(LiveTrader)

from LiveTrader import LiveTrader
from TradePosition import TradePosition
from TouchArea import TouchArea
from TradingStrategy import StrategyParameters, TouchDetectionAreas, TradingStrategy, is_security_shortable_and_etb, is_security_marginable, IntendedOrder
from TouchDetection import calculate_touch_detection_area, plot_touch_detection_areas, LiveTouchDetectionParameters
from TouchDetectionParameters import np_mean, np_median
from MultiSymbolDataRetrieval import retrieve_bar_data, retrieve_quote_data, fill_missing_data, get_data_with_retry, clean_quotes_data


import os, toml
from dotenv import load_dotenv

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'
ROUNDING_DECIMAL_PLACES = 10  # Choose an appropriate number of decimal places

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


debug = True

def debug_print(*args, **kwargs):
    if debug:
        print(*args, **kwargs)

In [17]:
async def main():
    # symbol = "AMZN"
    symbol = "AAPL"
    initial_balance = 10000
    # initial_balance = 10103.889074410155
    
    simulation_mode = True  # Set this to True for simulation, False for live trading

    touch_detection_params = LiveTouchDetectionParameters(
        symbol=symbol,
        atr_period=15,
        level1_period=15,
        multiplier=1.4,
        min_touches=3,
        # bid_buffer_pct=0.005,
        start_time=None,
        end_time='15:55',
        # end_time='11:20',
        use_median=True,
        touch_area_width_agg=np_median,

        ema_span=12,
        price_ema_span=26
        
    )

    strategy_params = StrategyParameters(
        initial_investment=initial_balance,
        do_longs=True,
        do_shorts=True,
        sim_longs=True,
        sim_shorts=True,
        
        use_margin=True,
        
        times_buying_power=1,
        
        soft_start_time = None, 
        soft_end_time = '15:50',
    )

    trader = LiveTrader(API_KEY, API_SECRET, symbol, initial_balance, touch_detection_params, strategy_params, simulation_mode)

    try:
        # await trader.run()
        
        # date_to_simulate = date(2024, 8, 20)
        date_to_simulate = date(2024, 9, 4)
        results = await trader.run_day_sim(date_to_simulate)
        print(results)
        
    except KeyboardInterrupt:
        print("Keyboard interrupt received. Stopping trader.")
    except Exception as e:
        print(f"{type(e).__qualname__}: {e}")
    finally:
        await trader.close()
        print("Trader stopped.")

In [18]:
import tracemalloc
tracemalloc.start()


import logging
# logging.getLogger('alpaca').setLevel(logging.DEBUG)
# logging.getLogger('LiveTrader').setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG)

# Run the main function
clear_output(wait=True)  # Clear previous output

# Check if we're already in an event loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    # If we're not in an event loop, create one and run our main function
    asyncio.run(main())
else:
    # If we're already in an event loop, create a task
    loop.create_task(main())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.alpaca.markets:443
DEBUG:urllib3.connectionpool:https://data.alpaca.markets:443 "GET /v2/stocks/AAPL/bars?start=2024-09-04T04%3A00%3A00%2B00%3A00&end=2024-09-05T04%3A00%3A00%2B00%3A00&timeframe=1Min&adjustment=raw HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://paper-api.alpaca.markets:443 "GET /v2/calendar?start=2024-09-04&end=2024-09-04 HTTP/1.1" 200 130
DEBUG:urllib3.connectionpool:https://paper-api.alpaca.markets:443 "GET /v2/assets/AAPL HTTP/1.1" 200 397
DEBUG:urllib3.connectionpool:https://paper-api.alpaca.markets:443 "GET /v2/assets/AAPL HTTP/1.1" 200 397


AAPL is marginable.
AAPL is shortable and ETB.


  0%|          | 0/958 [00:00<?, ?it/s]2024-10-28 17:14:57,157 - LiveTrader - INFO - Gap filled. Data is now ready for trading.
INFO:LiveTrader:Gap filled. Data is now ready for trading.
2024-10-28 17:14:57,159 - LiveTrader - INFO - Continuous data range: 2024-09-04 04:00:00-04:00 to 2024-09-04 04:02:00-04:00
INFO:LiveTrader:Continuous data range: 2024-09-04 04:00:00-04:00 to 2024-09-04 04:02:00-04:00
DEBUG:urllib3.connectionpool:https://data.alpaca.markets:443 "GET /v2/stocks/AAPL/quotes?start=2024-09-04T08%3A01%3A58%2B00%3A00&end=2024-09-04T08%3A02%3A01%2B00%3A00 HTTP/1.1" 200 None
  0%|          | 1/958 [00:00<02:23,  6.66it/s]DEBUG:urllib3.connectionpool:https://data.alpaca.markets:443 "GET /v2/stocks/AAPL/quotes?start=2024-09-04T08%3A02%3A58%2B00%3A00&end=2024-09-04T08%3A03%3A01%2B00%3A00 HTTP/1.1" 200 54
2024-10-28 17:14:57,320 - LiveTrader - ERROR - KeyError in execute_trading_logic at 2024-09-04 04:03:00-04:00: 'Requested level (symbol) does not match index name (None)'
ERROR:L

KeyError: 'Requested level (symbol) does not match index name (None)'
Trader stopped.
